In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical


# from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Convolution2D , MaxPooling2D, Activation

In [ ]:
trainset = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
testset = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
trainset

In [ ]:
xtrain = trainset.drop(columns = 'label').values.astype('float32')
ytrain = trainset['label'].values.astype('int32')
xtest = testset.values.astype('float32')

xtrain = xtrain.reshape(xtrain.shape[0], 28, 28,1)
xtest = xtest.reshape(xtest.shape[0],28,28,1)
ytrain


In [ ]:
def standardise(x):
   return (x-xtrain.mean())/xtrain.std()

In [ ]:
ytrain = to_categorical(ytrain)

np.random.seed(42)

In [ ]:
from sklearn.model_selection import train_test_split
gen = ImageDataGenerator()
x = xtrain
y = ytrain
xtrain,xval, ytrain, yval = train_test_split(xtrain, ytrain,test_size = 0.10, random_state = 42)
batches=gen.flow(xtrain,ytrain,batch_size = 64)
valbatches = gen.flow(xtrain,ytrain,batch_size = 64)




In [ ]:
model = Sequential([
    Lambda(standardise,input_shape=(28,28,1)),
    Dropout(0.2),
    Convolution2D(32,(3,3), activation = 'relu'),
    BatchNormalization(axis = 1),
    Convolution2D(32,(3,3),activation = 'relu'),
    MaxPooling2D(),
    BatchNormalization(axis = 1),
    Convolution2D(64,(3,3),activation = 'relu'),
    BatchNormalization(axis=1),
    Convolution2D(64,(3,3),activation = 'relu'),
    MaxPooling2D(),
    Convolution2D(128,(3,3),activation='relu'),
    MaxPooling2D(),
    Flatten(),
    BatchNormalization(),
    Dense(512,activation = 'relu'),
    BatchNormalization(),
    Dense(256,activation = 'relu'),
    BatchNormalization(),
    
    Dense(10, activation = 'softmax'),
])
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.optimizer.lr=0.05
history=model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, validation_data=valbatches, validation_steps=valbatches.n)


In [73]:
batches=gen.flow(x,y,batch_size = 64)
history=model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1)
pred = model.predict_classes(xtest,verbose = 0)
submissions=pd.DataFrame({"ImageId": list(range(1,len(pred)+1)),
                         "Label": pred})
submissions.to_csv("DR.csv", index=False, header=True)

Train for 16271 steps
  255/16271 [..............................] - ETA: 2:50 - loss: 0.1184 - accuracy: 0.9676

NameError: name 'predictions' is not defined